# beomi/KcELECTRA-base

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 700753012877022373
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2254123828
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2939343429369144338
physical_device_desc: "device: 0, name: GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [2]:
%run KcELECTRA-base.version.py
!pip freeze

라이브러리 설치가 완료되었습니다.
absl-py==2.0.0
accelerate==0.20.1
astunparse==1.6.3
attrs==23.1.0
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backports.functools_lru_cache_1687772187254/work
beautifulsoup4==4.12.2
bleach==6.0.0
cached-property==1.5.2
cachetools==5.3.2
certifi==2023.7.22
charset-normalizer==3.3.1
colorama @ file:///home/conda/feedstock_root/build_artifacts/colorama_1666700638685/work
cycler==0.11.0
debugpy @ file:///D:/bld/debugpy_1660619187469/work
decorator @ file:///home/conda/feedstock_root/build_artifacts/decorator_1641555617451/work
defusedxml==0.7.1
emoji==1.2.0
entrypoints @ file:///home/conda/feedstock_root/build_artifacts/entrypoints_1643888246732/work
et-xmlfile==1.1.0
exceptiongroup==1.1.3
fastjsonschema==2.18.1
filelock==3.12.2
flatbuffers==1.12
fonttools==4.38.0
fsspec==2023.1.0
gast==0.4.0
google-auth==2.23.3
google-auth-oauthlib==0.4.6
google

In [3]:
import re
import torch
import pandas as pd

# deivce 선택
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("devcie:", device)

c:\Users\GJAISCHOOL\.conda\envs\nv37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


devcie: cpu


In [4]:
# 엑셀 파일에서 데이터프레임 읽기
df = pd.read_excel(r"C:\Users\GJAISCHOOL\Desktop\X_filter\Algorithm\dataset\asdf.xlsx")

# 'Sentence' 칼럼의 값을 문자열로 변환하여 리스트로 저장
str_data = df['Sentence'].astype(str).tolist()

# 'Sentence' 칼럼의 값을 하나의 문자열로 결합
all_sentences = ' '.join(str_data)

print(df.shape)
df.head()

(96, 2)


,Sentence,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1


In [5]:
null_idx = df[df.label.isnull()].index                             # 해당 index에 null 값 확인
df.loc[null_idx, "Sentence"]                                       # null 값이 존재한 인덱스의 content 값 불러오기

# lable은 content의 가장 끝 문장열로 설정
df.loc[null_idx, "label"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정
df.loc[null_idx, "Sentence"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-2])

In [6]:
train_data = df.sample(frac=0.8, random_state=42)                 # train(80%), test(20%) 셋 구분 
test_data = df.drop(train_data.index)                             # 랜덤으로 샘플링(랜덤으로 숫자 배치)

# 데이터셋 갯수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=["Sentence"], inplace=True)
test_data.drop_duplicates(subset=["Sentence"], inplace=True)

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))

중복 제거 전 학습 데이터셋 : 77
중복 제거 전 테스트 데이터셋 : 19
중복 제거 후 학습 데이터셋 : 77
중복 제거 후 테스트 데이터셋 : 19


In [7]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, TFElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# KcELECTRA 모델 및 토크나이저 로드
model_name = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
tokenized_train_sentences = tokenizer(
    list(train_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [9]:
tokenized_test_sentences = tokenizer(
    list(test_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [10]:
print(tokenized_train_sentences[0])        # 0번째 문장에 해당하는 객체 
print(tokenized_train_sentences[0].tokens) # 0번째 문장에 토큰의 목록
print(tokenized_train_sentences[0].ids)    # 0번째 문장에 대한 고유 ID

Encoding(num_tokens=72, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['Ġìļ°ë¦¬ëĤĺëĿ¼', 'ìĺĢìľ¼ë©´', 'ĠëĤ¨ìĦ±', '1', 'ëħĦ', 'Ġì§ĳ', 'íķ´', 'ìľł', 'ìĺĪ', ',', 'ĠìĹĦë§Ī', 'Ġ20', 'ëħĦ', 'ĠëķĮë¦´', 'ê²ĥê°ĻëĦ¤ìļĶ', '.', 'ĠìķĦëĭĺ', 'ë§Ĳ', 'êµ¬', '...', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[1158, 6167, 4875, 21, 604, 639, 305, 608, 1193, 16, 3291, 1321, 604, 27887, 28472, 18, 1852, 473, 480, 351, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [11]:
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encoding, labels):
        self.encodings = encoding
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
        item["labels"] = torch.tensor(self.labels[idx])                             # key: toch.tensor(val[ix])라는 item 딕셔너리 형성
        return item                                                                 # 새로운 labels 키 값에 value torch.tensor(self.labels[idx]) 쌍을 생성
    
    def __len__(self):
        return len(self.labels)                                                     # self.labels 길이 반환

In [12]:
# train_set, test_set에 대한 데이터셋을 각각 생성

train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)   # 사전 학습된 모델 찾아오기
model.to(device)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.de

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [14]:
# 3 - 2 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir = './results',           # 학습결과 저장경로
    num_train_epochs=10,                # 학습 epoch 설정
    per_device_train_batch_size = 8,    # train batch_size 설정
    per_device_eval_batch_size = 64,    # test batch_size 설정
    logging_dir = './logs',             # 학습 log 저장경로
    logging_steps=100,                  # 학습 log 기록 단위
    save_total_limit = 2,               # 학습결과 저장 최대갯수
)

In [15]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 학습과정에서 사용할 평가지표를 위한 함수 설정
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # 정밀도, 재현율, f1 구하기 
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  # 정확도 구하기
  acc = accuracy_score(labels, preds)
  return{
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [16]:
# Trainer 모듈을 사용해 모델의 학습을 컨트롤하은 trainer를 생성
trainer = Trainer(
    model = model,                       # 학습하고자하는 Transformers model
    args=training_args,                  # 위에서 정의한 Trainging Arguments
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=test_dataset,           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가지표
)

In [17]:
trainer.train()

c:\Users\GJAISCHOOL\.conda\envs\nv37\lib\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\GJAISCHOOL\.conda\envs\nv37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
100%|██████████| 100/100 [04:58<00:00,  2.98s/it]

{'loss': 0.2692, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 298.413, 'train_samples_per_second': 2.58, 'train_steps_per_second': 0.335, 'train_loss': 0.26918706893920896, 'epoch': 10.0}


TrainOutput(global_step=100, training_loss=0.26918706893920896, metrics={'train_runtime': 298.413, 'train_samples_per_second': 2.58, 'train_steps_per_second': 0.335, 'train_loss': 0.26918706893920896, 'epoch': 10.0})

In [18]:
# 평가를 수행하고 custom_ 접두사가 붙은 평가 지표를 출력
results = trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix='custom_')

print(results)

c:\Users\GJAISCHOOL\.conda\envs\nv37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
100%|██████████| 1/1 [00:00<00:00, 83.33it/s]

{'custom__loss': 0.8951232433319092, 'custom__accuracy': 0.631578947368421, 'custom__f1': 0.5882352941176471, 'custom__precision': 0.5555555555555556, 'custom__recall': 0.625, 'custom__runtime': 0.915, 'custom__samples_per_second': 20.765, 'custom__steps_per_second': 1.093, 'epoch': 10.0}


In [19]:
# 0: curse, 1: non_curse
def sentence_predict(sent):
    # 평가모드로 변경
    model.eval()

    # 입력된 문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 모델이 위치한 GPU로 이동
    # tokenized_sent.to(deivce)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
        )

    # 결과 return
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    sentence = True
    input_sentence = tokenized_sent["input_ids"]
    if result == 0:
        sentence = True

    elif result == 1:
        sentence = False
    return sentence, input_sentence


sentence_predict("씨발이게 맞아?")

(True, tensor([[2295,  640, 3898, 5170,   33]]))

In [20]:
def found_word(input_sentence, found_bad_word):
    result = input_sentence
    if found_bad_word:
        result = badword_find(result)
    return result

def badword_find(input_sentence):
    result = input_sentence
    badword_df = pd.read_excel(r'C:\Users\GJAISCHOOL\Desktop\X_filter\Algorithm\dataset\word_list.xlsx')
    
    found_bad_word = False  # 입력 문장에 단어가 발견되었는지를 나타내는 플래그
    for idx, row in badword_df.iterrows():
        if row["WORD"] in input_sentence:
            # 'WORD'가 입력 문장에 포함된 경우
            new_word = row["대체어"]
            if not pd.isnull(new_word):
                result = result.replace(row["WORD"], new_word)
                found_bad_word = True
                break  # 대체어를 찾았으므로 반복문 종료
            else:
                # result = result.replace(row["WORD"], "*" * len(row["WORD"]))
                result = "욕설을 사용했습니다."
                found_bad_word = True
    
    if not found_bad_word:
        # result = "@" * len(input_sentence)
        result = "혐오 표현입니다."
    return result

# 테스트
input_sentence = "아가리"
speak_result = badword_find(input_sentence)
print(speak_result)

입


In [21]:
badword_find(input_sentence)

'입'

In [22]:
def speak(sent):
    sentence = sentence_predict(sent)
    if sentence[0] == False:
        return sent
    elif sentence[0] == True:
        return badword_find(sent)

In [23]:
import kss
from pykospacing import Spacing
from soynlp.normalizer import repeat_normalize

# 띄어쓰기 설정
spacing = Spacing()

In [24]:
# 특수문자 제거
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ|a-zA-Z0-9]','', text)
    return text

In [25]:
def clean_and_repeat_normalize(text):
    cleansed_text = cleanse(text)                                             # 특수문자 제거
    normalized_text = repeat_normalize(cleansed_text, num_repeats=2)          # 중복문자 제거
    input_data = re.sub(r'\d', '', normalized_text)                           # 숫자 제거
    normalized_text = spacing(input_data)                                     # 띄어쓰기 보정 

    return normalized_text

clean_and_repeat_normalize("어바지가방에들어가신다")

'어바지가 방에 들어가신다'

In [29]:
def final_output():
    input_text = clean_and_repeat_normalize(input())
    sentences = kss.split_sentences(input_text)

    sentences_list = []
    for sentence in sentences:
        text_output = speak(sentence)
        sentences_list.append(text_output)
        print(sentence)

    long_test = ' '.join(sentences_list)
    print(long_test)
    return long_test

In [30]:
final_output()

씨발너가 그 따구로 사니까 안 되는 거잖아
그냥 넌 나가 죽어야 해 엄마가 너태어난 날 울겠다
씨발너가 그 따구로 사니까 안 되는 거잖아 그냥 넌 나가 죽어야 해 엄마가 너태어난 날 울겠다


'씨발너가 그 따구로 사니까 안 되는 거잖아 그냥 넌 나가 죽어야 해 엄마가 너태어난 날 울겠다'

In [32]:
spacing("씨발너가말야?")

'씨발너가 말야?'